In [1]:
import json
from collections import OrderedDict
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import clear_output

In [2]:
with open("test-v2.0.en.json") as f:
    f_ = json.load(f, object_hook=OrderedDict)
with open("test-v2.0.cs.raw.json") as f:
    f = json.load(f, object_hook=OrderedDict)

In [3]:
def _show_dialog(tooltip, text, callback):
    print(tooltip)
    u = widgets.Textarea(
        value=text,
        description='Correct:',
        disabled=False,
        layout=widgets.Layout(width='100%', height='300px')
    )
    #w = widgets.Text(
    #    value=context,
    #    description='Paste here',
    #    disabled=False,
    #    layout=widgets.Layout(width='100%', height='50px')
    #)

    button = widgets.Button(description="Next")
    button._dirtyhack = u
    output = widgets.Output()

    display(u)
    #display(w)
    display(button, output)
    button.on_click(callback)

In [4]:
#i, j = 0, 0
#state = "context"

def process_next(i, j, state):
    
    if state == 'context':
        annotate_context(i, j, state)
    elif state.startswith('question'):
        q = int(state.split("_")[1])
        annotate_question(i, j, q)
    elif state.startswith('answers'):
        q = int(state.split("_")[1])
#         a = int(state.split("_")[2])
        annotate_answers(i, j, q)

def annotate_context(i, j, state):
    print(i, "/", len(f['data']), ", ", j, "/", len(f['data'][i]['paragraphs']))
    context = f['data'][i]['paragraphs'][j]['context']
    tooltip = "Please correct this text:\nOriginal: %s" % f_['data'][i]['paragraphs'][j]['context']
    def callback(w):
        f['data'][i]['paragraphs'][j]['context'] = w._dirtyhack.value
        clear_output()
#             print(w._dirtyhack.value)
        process_next(i, j, state="question_0")
    _show_dialog(tooltip, context, callback)

def annotate_question(i, j, q):
    question = f['data'][i]['paragraphs'][j]['qas'][q]['question']
    question_en = f_['data'][i]['paragraphs'][j]['qas'][q]['question']
    tooltip = "Please correct this question:\nOriginal:%s" % (question_en)
    
    def callback(w):
        f['data'][i]['paragraphs'][j]['qas'][q]['question'] = w._dirtyhack.value
        clear_output()
        process_next(i, j, state="answers_%d" % q)
        return  # TODO
    
    _show_dialog(tooltip, text=question, callback=callback)


def annotate_answers(i, j, q):  # TODO: show original span
    print("Please provide exactly 4 answers to the question.")
    
    if f['data'][i]['paragraphs'][j]['qas'][q]['is_impossible']:
        # TODO: edit plausible answers
        print("Warning: impossible answer")
    
    question = f['data'][i]['paragraphs'][j]['qas'][q]['question']
    question_en = f_['data'][i]['paragraphs'][j]['qas'][q]['question']
    context = f['data'][i]['paragraphs'][j]['context']  # now corrected
    answer = f_['data'][i]['paragraphs'][j]['qas'][q]['answers']
    tooltip = "Please mark answer 1 with (text)@@@(answer)@@@(text)\nQuestion:%s\nOriginal:%s\nAnswer:%s" % (question, question_en, answer)
    
    def callback(w):
        # TODO: find span marked by user
        start_idx = w._dirtyhack.value.find("@@@") + 3
        end_idx = w._dirtyhack.value.find("@@@", start_idx+3)
        
        if start_idx == -1 or end_idx == -1:
            w._dirtyhack.value = f['data'][i]['paragraphs'][j]['context']
            print("Invalid answer! Please try again, context was reset.")
            return
        
        if not hasattr(w, "answer_idx"):
            answer_idx = 0
        else:
            answer_idx = w.answer_idx
        # TODO: save ith answer
        answer_idx += 1
        w.answer_idx = answer_idx
        # TODO: show i+1th dialog
        if answer_idx < 4:
            # TODO: say which answer it is
            print("Editing answer %d" % answer_idx)
            w._dirtyhack.value = f['data'][i]['paragraphs'][j]['context']
            #_show_dialog(tooltip, text=context, callback=callback)
        else:
            clear_output()
            # TODO: loop answer
            if q + 1 < len(f['data'][i]['paragraphs'][j]['qas']):
                process_next(i, j, "question_%d" % (q + 1))
            elif j + 1 < len(f['data'][i]['paragraphs']):
                process_next(i, j+1, "context")
            elif i + 1 < len(f['data']):
                process_next(i+1, 0, "context")
            else:
                print("Done!")
        
        #print(start_idx, end_idx)
        #print(w._dirtyhack.value[start_idx:end_idx])        
        return
    
    _show_dialog(tooltip, text=context, callback=callback)
    print("Editing answer %d" % 0)


## Annotation

In [5]:
process_next(i=0, j=0, state="context")

Please provide exactly 4 answers to the question.
Please mark answer 1 with (text)@@@(answer)@@@(text)
Question:Kdy byli Normani v Normandii?
Original:When were the Normans in Normandy?
Answer:[OrderedDict([('text', '10th and 11th centuries'), ('answer_start', 94)]), OrderedDict([('text', 'in the 10th and 11th centuries'), ('answer_start', 87)]), OrderedDict([('text', '10th and 11th centuries'), ('answer_start', 94)]), OrderedDict([('text', '10th and 11th centuries'), ('answer_start', 94)])]


Textarea(value='Normané ( Norman : Nourmands ; francouzština : Normandi ; latina : Normanni ) byli lidé, kteří…

Button(description='Next', style=ButtonStyle())

Output()

Editing answer 0


In [119]:
f['data'][0]['paragraphs'][0]['context']

'Normané ( Norman : Nourmands ; francouzština : Normandi ; latina : Normané ) byli lidé, kteří v 10. a 11. století dali své jméno Normandii , oblasti ve Francii . Byli potomky severských nájezdníků a pirátů z Dánska, Islandu a Norska, kteří pod vedením svého vůdce Rolla souhlasili s přísahou věrnosti králi Karlu III. ze Západní Francie. Prostřednictvím generací asimilace a mísení s původními Franky a Římsko-Gaulishskými populacemi by jejich potomci postupně splynuli s karolínskými kulturami Západní Francie. Výrazná kulturní a etnická identita Normanů se objevila zpočátku v první polovině 10. století a v následujících stoletích se dále vyvíjela .'

In [15]:
f_['data'][0]['paragraphs'][0]['context']

'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.'

In [51]:
f['data'][0]['paragraphs'][0]['qas'][0], f['data'][0]['paragraphs'][0]['qas'][-1]

(OrderedDict([('question', 'V jaké zemi se nachází Normandie?'),
              ('id', '56ddde6b9a695914005b9628'),
              ('answers',
               [OrderedDict([('text', 'Francii'), ('answer_start', 152)]),
                OrderedDict([('text', 'Francii'), ('answer_start', 152)]),
                OrderedDict([('text', 'Francii'), ('answer_start', 152)]),
                OrderedDict([('text', 'Francii'), ('answer_start', 152)])]),
              ('is_impossible', False)]),
 OrderedDict([('plausible_answers',
               [OrderedDict([('text', '10. století'),
                             ('answer_start', 594)])]),
              ('question', 'Kdy se objevila identita Franků?'),
              ('id', '5ad39d53604f3c001a3fe8d4'),
              ('answers', []),
              ('is_impossible', True)]))

In [6]:
n_contexts = len(f['data'])
n_questions = sum(len(f_['data'][i]['paragraphs'][0]['qas']) for i in range(len(f_['data'])))
n_answers = n_questions * 4
print("To annotate: %d questions, %d answers in %d contexts" % (n_questions, n_answers, n_contexts))

To annotate: 111 questions, 444 answers in 10 contexts


In [17]:
with open("test-v2.0.cs.annot.json", "w") as fp:
    json.dump(f, fp)

In training : 412 questions, 1648 answers in 35 contexts


In [12]:
3860 * 4

15440

### obtain all questions (EN)

In [2]:
squad_questions = []

with open("../squad2/train-v2.0.json") as fp:
    train = json.load(fp)
for i in train['data']:
    for j in i['paragraphs']:
        # one context
        context = j['context']
        for k in j['qas']:
            # one question
            question = k['question']
            squad_questions.append(question)

with open("../squad2/questions.txt", "w") as fp:
    for question in squad_questions:
        fp.write(question + "\n")
print("Wrote %d questions" % len(squad_questions))

FileNotFoundError: [Errno 2] No such file or directory: '../squad2/train-v1.1.json'

In [7]:
import re
import numpy as np
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
tokenize = TreebankWordTokenizer().tokenize

def hood(context, idx):
    text = context[idx-80:idx+80]
    start = text.find(" ")
    end = text.rfind(" ", start + 1)
    text = text[start:end]
    return re.sub('[?!,.]', '', text)

stop_words = set(stopwords.words('english'))

def overlap(a, b):
    # remove stopwords
    a = set(a) - stop_words
    b = set(b) - stop_words
    # TODO: fraction?
    return len(a & b)

In [31]:
overlaps = []

with open("../squad2/dev-v2.0.json") as fp:
    train = json.load(fp)
for i in train['data']:
    for j in i['paragraphs']:
        # one context
        context = j['context']
        for k in j['qas']:
            # one question
            question = k['question']
            if not k['answers']: continue
            answer = hood(context, k['answers'][0]['answer_start'])
            if not answer: continue
            #print(answer)
            o = overlap(tokenize(question), tokenize(answer))
            overlaps.append((o, question, answer))

In [32]:
print(sum(1 for i in overlaps if i[0] == 0))
print("Mean: %d, Max: %d overlap" % (np.mean([o[0] for o in overlaps]), np.max([o[0] for o in overlaps])))

411
Mean: 2, Max: 15 overlap


In [37]:
print([o for o in overlaps if o[0] == 0][:2])

[(0, 'What religion were the Normans', ' spirit and eventually for their Christian piety becoming exponents of the Catholic orthodoxy into which they assimilated They adopted the Gallo-Romance'), (0, 'What is the original meaning of the word Norman?', ' or Nordmannus (recorded in Medieval Latin 9th century) to mean "Norseman')]


In [139]:
# Bolo ich 14 807 s nulovym overlapom +-40 chars, ale bacha - slova mali rovnake lemmy, preto lemmatizujem!

In [9]:
from nltk.stem import LancasterStemmer
def stem(seq):
    stemmer = LancasterStemmer()
    return [stemmer.stem(word) for word in seq]

In [42]:
overlaps_lemma = []

with open("../squad2/dev-v2.0.json") as fp:
    train = json.load(fp)
for ii, i in enumerate(train['data']):
    for jj, j in enumerate(i['paragraphs']):
        # one context
        context = j['context']
        for kk, k in enumerate(j['qas']):
            # one question
            question = k['question']
            if not k['answers']: continue
            answer = hood(context, k['answers'][0]['answer_start'])
            if not answer: continue
            #print(answer)
            o = overlap(stem(tokenize(question)), stem(tokenize(answer)))
            overlaps_lemma.append((o, question, answer, (ii, jj, kk)))

In [60]:
print("Mean: %d, Max: %d overlap" % (np.mean([o[0] for o in overlaps_lemma]), np.max([o[0] for o in overlaps_lemma])))
print("Mean tokens: %d" % np.mean([len(set(tokenize(o[2])) - stop_words) for o in overlaps_lemma]))
print(sum(1 for i in overlaps_lemma if i[0] == 0))
print(sum(1 for i in overlaps_lemma if i[0] > 6))

Mean: 3, Max: 11 overlap
Mean tokens: 14
218
193


In [45]:
print([o for o in overlaps_lemma if o[0] == 0][:2])

[(0, 'What religion were the Normans', ' spirit and eventually for their Christian piety becoming exponents of the Catholic orthodoxy into which they assimilated They adopted the Gallo-Romance', (0, 1, 2)), (0, "Who ruined Roussel de Bailleul's plans for an independent state?", ' support from the local population but he was stopped by the Byzantine general Alexius', (0, 8, 2))]


In [64]:
print([o for o in overlaps_lemma if o[0] == 10][:2])

[(10, 'The term "southern" California usually refers to how many of the southern-most counties of the state?', ' and southern California) the term "southern California" usually refers to the ten southern-most counties of the state This definition coincides neatly with', (2, 9, 2)), (10, 'What was the one linear HD channel Virgin Media carried from November 2006 to July 2009?', ' 30 November 2006 until 30 July 2009 it only carried one linear HD channel BBC HD after the conclusion of the ITV HD trial Virgin Media has claimed that', (3, 18, 3))]


In [133]:
# TODO: make a dataset of them!

In [65]:
print(len(overlaps_lemma))

4914


### Collect datasets with:
- minimal overlap
- maximal overlap

In [52]:
import copy

low_overlap = [o[-1] for o in overlaps_lemma if o[0] == 0]
big_overlap = [o[-1] for o in overlaps_lemma if o[0] > 6]


with open("../squad2/dev-v2.0.json") as fp:
    train = json.load(fp)

easy_dataset = {"version": train['version'], "data": []}
hard_dataset = {"version": train['version'], "data": []}

def add_to(dataset, topic, para, qa):
    if not dataset['data'] or (dataset['data'][-1]['title'] != topic['title']):
        # add context
        dataset['data'].append({"title": topic['title'], "paragraphs": []})
    if not dataset['data'][-1]['paragraphs'] or dataset['data'][-1]['paragraphs'][-1]['context'] != para['context']:
        dataset['data'][-1]['paragraphs'].append({"context": para['context'], "qas": []})
    if not dataset['data'][-1]['paragraphs'][-1]['qas'] or dataset['data'][-1]['paragraphs'][-1]['qas'][-1]['question'] != qa['question']:
        dataset['data'][-1]['paragraphs'][-1]['qas'].append(qa)

for ii, i in enumerate(train['data']):
    for jj, j in enumerate(i['paragraphs']):
        # one context
        context = j['context']
        for kk, k in enumerate(j['qas']):
            # one question
            if (ii, jj, kk) in low_overlap:
                add_to(hard_dataset, i, j, k)
            if (ii, jj, kk) in big_overlap:
                add_to(easy_dataset, i, j, k)

In [55]:
with open("test-en-easy.json", "w") as fp:
    json.dump(easy_dataset, fp)
with open("test-en-hard.json", "w") as fp:
    json.dump(hard_dataset, fp)

In [2]:
# Check if similar for translated test-de-easy and test-de-hard

In [11]:
overlaps_lemma = []

with open("test-en-easy.json") as fp:
    train = json.load(fp)
for ii, i in enumerate(train['data']):
    for jj, j in enumerate(i['paragraphs']):
        # one context
        context = j['context']
        for kk, k in enumerate(j['qas']):
            # one question
            question = k['question']
            if not k['answers']: continue
            answer = hood(context, k['answers'][0]['answer_start'])
            if not answer: continue
            #print(answer)
            o = overlap(stem(tokenize(question)), stem(tokenize(answer)))
            overlaps_lemma.append((o, question, answer, (ii, jj, kk)))

print("-- ENGLISH EASY DATASET --")
print("Mean: %d, Max: %d overlap" % (np.mean([o[0] for o in overlaps_lemma]), np.max([o[0] for o in overlaps_lemma])))
print("Mean tokens: %d" % np.mean([len(set(tokenize(o[2])) - stop_words) for o in overlaps_lemma]))
print(sum(1 for i in overlaps_lemma if i[0] == 0))
print(sum(1 for i in overlaps_lemma if i[0] > 6))

-- ENGLISH EASY DATASET --
Mean: 7, Max: 11 overlap
Mean tokens: 15
0
193


In [12]:
overlaps_lemma = []

with open("test-de-easy.json") as fp:
    train = json.load(fp)
for ii, i in enumerate(train['data']):
    for jj, j in enumerate(i['paragraphs']):
        # one context
        context = j['context']
        for kk, k in enumerate(j['qas']):
            # one question
            question = k['question']
            if not k['answers']: continue
            answer = hood(context, k['answers'][0]['answer_start'])
            if not answer: continue
            #print(answer)
            o = overlap(stem(tokenize(question)), stem(tokenize(answer)))
            overlaps_lemma.append((o, question, answer, (ii, jj, kk)))

print("-- GERMAN EASY DATASET --")
print("Mean: %d, Max: %d overlap" % (np.mean([o[0] for o in overlaps_lemma]), np.max([o[0] for o in overlaps_lemma])))
print("Mean tokens: %d" % np.mean([len(set(tokenize(o[2])) - stop_words) for o in overlaps_lemma]))
print(sum(1 for i in overlaps_lemma if i[0] == 0))
print(sum(1 for i in overlaps_lemma if i[0] > 6))

-- GERMAN EASY DATASET --
Mean: 6, Max: 13 overlap
Mean tokens: 19
3
79


In [13]:
overlaps_lemma = []

with open("test-en-hard.json") as fp:
    train = json.load(fp)
for ii, i in enumerate(train['data']):
    for jj, j in enumerate(i['paragraphs']):
        # one context
        context = j['context']
        for kk, k in enumerate(j['qas']):
            # one question
            question = k['question']
            if not k['answers']: continue
            answer = hood(context, k['answers'][0]['answer_start'])
            if not answer: continue
            #print(answer)
            o = overlap(stem(tokenize(question)), stem(tokenize(answer)))
            overlaps_lemma.append((o, question, answer, (ii, jj, kk)))

print("-- ENGLISH HARD DATASET --")
print("Mean: %d, Max: %d overlap" % (np.mean([o[0] for o in overlaps_lemma]), np.max([o[0] for o in overlaps_lemma])))
print("Mean tokens: %d" % np.mean([len(set(tokenize(o[2])) - stop_words) for o in overlaps_lemma]))
print(sum(1 for i in overlaps_lemma if i[0] == 0))
print(sum(1 for i in overlaps_lemma if i[0] > 6))

-- ENGLISH HARD DATASET --
Mean: 0, Max: 0 overlap
Mean tokens: 14
218
0


In [14]:
overlaps_lemma = []

with open("test-de-hard.json") as fp:
    train = json.load(fp)
for ii, i in enumerate(train['data']):
    for jj, j in enumerate(i['paragraphs']):
        # one context
        context = j['context']
        for kk, k in enumerate(j['qas']):
            # one question
            question = k['question']
            if not k['answers']: continue
            answer = hood(context, k['answers'][0]['answer_start'])
            if not answer: continue
            #print(answer)
            o = overlap(stem(tokenize(question)), stem(tokenize(answer)))
            overlaps_lemma.append((o, question, answer, (ii, jj, kk)))

print("-- GERMAN HARD DATASET --")
print("Mean: %d, Max: %d overlap" % (np.mean([o[0] for o in overlaps_lemma]), np.max([o[0] for o in overlaps_lemma])))
print("Mean tokens: %d" % np.mean([len(set(tokenize(o[2])) - stop_words) for o in overlaps_lemma]))
print(sum(1 for i in overlaps_lemma if i[0] == 0))
print(sum(1 for i in overlaps_lemma if i[0] > 6))

-- GERMAN HARD DATASET --
Mean: 0, Max: 5 overlap
Mean tokens: 18
76
0


In [2]:
## Create HTML

In [8]:
def html_from_file(file, lang='de'):
    
    if lang == 'de':
        paragraph_key = "Absatz"
        question_key = "Frage"
        answer_key = "Mogliche antwort"
    elif lang == 'en':
        paragraph_key = "Paragraph"
        question_key = "Question"
        answer_key = "Possible answer"
    
    with open(file + ".json") as fp:
        train = json.load(fp)

    doc = []

    for ii, i in enumerate(train['data']):
        doc.append("<h2>%s</h2>\n" % i['title'])
        for jj, j in enumerate(i['paragraphs']):
            doc.append("<h3>%s %d.%d</h3>\n" % (paragraph_key, ii, jj))
            # one context
            context = j['context']
            doc.append("<p>%s</p><br>\n" % context)
            doc.append("<table border=\"1\">\n")
            for kk, k in enumerate(j['qas']):
                # one question
                question = k['question']
                doc.append("<tr><td>%s</td><td>%s</td></tr>\n" % (question_key, question))
                if not k['answers']: continue
                answers = k['answers']
                for answer in answers:
                    doc.append("<tr><td>%s</td><td>%s</td></tr>\n" % (answer_key, answer['text']))
            doc.append("</table>\n")
            doc.append("<hr>\n")

    doc = "".join(doc)
    
    with open(file + "-in.html", "w") as fp:
        fp.write("<html>\n<body>\n" + doc + "</body>\n</html>\n")

In [10]:
html_from_file("test-de-easy")
html_from_file("test-de-hard")
html_from_file("test-en-easy", 'en')
html_from_file("test-en-hard", 'en')